In [1]:
%matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.core.display import display, HTML
from biosppy.signals import ecg
# Create high pass filter
from scipy import signal
from scipy import stats
#Import the SMF module for multivariate
import statsmodels.formula.api as smf
ax = plt.gca()

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Using matplotlib backend: MacOSX


In [2]:
import io
import zipfile
import json

def bland_altman_plot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    plt.scatter(mean, diff, *args, **kwargs)
    plt.axhline(md,           color='darkblue', linestyle='-')
    plt.axhline(md + 1.96*sd, color='gray', linestyle='--')
    plt.axhline(md - 1.96*sd, color='gray', linestyle='--')
    
def scatterplot(data1, data2, *args, **kwargs):
    data1     = np.asarray(data1)
    data2     = np.asarray(data2)
    mean      = np.mean([data1, data2], axis=0)
    diff      = data1 - data2                   # Difference between data1 and data2
    md        = np.mean(diff)                   # Mean of the difference
    sd        = np.std(diff, axis=0)            # Standard deviation of the difference

    plt.scatter(mean, diff, *args, **kwargs)
    
    
    
def zip_to_object(zip_bytes):

    zipf = zipfile.ZipFile(io.BytesIO(zip_bytes),"r");
    files = zipf.filelist
    meta = json.loads(zipf.read("meta.json"))

    for file in filter( lambda f: f.filename.endswith(".npy"),files):
        meta[file.filename[:-4]] = np.load(io.BytesIO(zipf.read(file.filename)))

    return meta

def clean_ppg_signal_from_artefact_beats(ppg_df):
    i = 0
    list = []
    #Print successive differences to filter out the bad signal
    df = pd.DataFrame(columns=['diff','location'])
    
    
    for ts in ppg_df["rr_raw"][1:,0]:
        #print(ppg_df["rr_raw"][i,0])
        diff = ts - ppg_df["rr_raw"][i,0]
        df.loc[i] = [diff, ts]
        #print(diff)
        i = i + 1
        list.append(diff)
 

    #Sampling rate is 100Hz
  
    median = df['diff'].median()
    #display(df["diff"].dtype)
    #Only allow segments of the signal that have RR intervals between 0.5 and 1.5 of the median RR interval; otherwise they are likely artefactual
    i = 0
    display("Nodifference, just crop 10 seconds")
    plot_chart(ppg_df)
    selectrow = 10000
    item = np.argmax(ppg_df["timestamps"] > (selectrow))
    ppg_df["signal_raw"] = ppg_df["signal_raw"][item:,:]
    ppg_df["signal_fixed"] = ppg_df["signal_fixed"][item:,:]
    ppg_df["signal_filtered"] = ppg_df["signal_filtered"][item:,:]
    ppg_df["timestamps"] = ppg_df["timestamps"][item:, :]
    print("Zero is the select row")
    plot_chart(ppg_df)

    #Since argmax will stop at the first True ("In case of multiple occurrences of the maximum values, the indices corresponding to the first occurrence are returned.") and doesn't save another list.        


    ppg_df["timestamps"] = ppg_df["timestamps"] - item

    ppg_df["rr_raw"] = ppg_df["rr_raw"][i:]
    ppg_df["rr_raw"] = ppg_df["rr_raw"] - item
    ppg_df["rr_filtered"] = ppg_df["rr_filtered"][i:]
    ppg_df["rr_filtered"] = ppg_df["rr_filtered"] - item
    
    plot_chart(ppg_df)
    print(len(ppg_df["signal_raw"]))
    return ppg_df


def plot_chart(ppg_df):

    #Plot the new chart with the filtered out signal
    plt.figure(figsize=(10,10))

    ax = plt.subplot(5,1,1)
    plt.plot(ppg_df["timestamps"],ppg_df["signal_raw"])
    plt.subplot(5,1,2,sharex=ax)
    plt.plot(ppg_df["timestamps"],ppg_df["signal_fixed"])
    plt.ylim((1,2.5))
    plt.subplot(5,1,3,sharex=ax)
    plt.plot(ppg_df["timestamps"],ppg_df["signal_filtered"])

    for ts in ppg_df["rr_raw"][:,0]:
        plt.axvline(x = ts,color="green")

    for ts in ppg_df["rr_filtered"][:,0]:
        plt.axvline(x = ts,color="g")


    plt.subplot(5,1,4,sharex=ax)
    plt.plot(ppg_df["rr_filtered"][:,0],ppg_df["rr_filtered"][:,1],"o")

    plt.show()

    #for key in ["bpm","sdsd","rmssd"]:
        #print(key, ppg_df[key])

    print(json.dumps(ppg_df["metadata"]["deviceDetails"],indent=2))
    return("Chart done")




In [3]:
heh_mrn_linkage = pd.read_csv("data/Azumio_log.csv", sep=",")
ecg_data = pd.read_csv("data/db_xml.txt", sep=",")
ecg_data = ecg_data.drop_duplicates(subset=['filename'], keep=False)
ecg_data['mrn'] = pd.to_numeric(ecg_data['mrn'], errors='coerce')
heh_mrn_linkage['mrn'] = pd.to_numeric(heh_mrn_linkage['mrn'], errors='coerce')
#heh_mrn_linkage = heh_mrn_linkage.drop_duplicates(subset=['mrn'], keep=False)

In [4]:
azumio_ecg_mrn_merged = pd.merge(ecg_data, heh_mrn_linkage, left_on='mrn', right_on="mrn", how='inner')
azumio_ecg_mrn_merged = azumio_ecg_mrn_merged.loc[:, ~azumio_ecg_mrn_merged.columns.str.match('Unnamed')]

In [5]:

## Delete these bad recording
poor_recordings = [255182, 256090, 257310, 257510, 257310, 41986]

for index, item in enumerate(poor_recordings):
    azumio_ecg_mrn_merged = azumio_ecg_mrn_merged[azumio_ecg_mrn_merged['user_id'] != item]





#display(azumio_ecg_mrn_merged.tail(n=30))

In [6]:
def read_signals(ppg_path, ecg_path, ppg_sample_rate, ppg_upsample_rate):
    with open(ppg_path,"rb") as f:
        d = f.read()
        if int(row['user_id'] in poor_recordings):
            print("Poor recording")
            ppg_df = zip_to_object(d)
            ppg_df = clean_ppg_signal_from_artefact_beats(ppg_df)
        else:
            ppg_df = zip_to_object(d)
            
            
    ecg_data = pd.read_csv(ecg_path, sep=",")
    #display(ecg_data[' V5'.head(n=5))
    
    ppg_sample_rate_check = int(np.round(1000.0/np.mean(ppg_df["timestamps"][1:]-ppg_df["timestamps"][:-1])))
    if ppg_sample_rate_check != ppg_sample_rate:
        display("ppg_sample_rate_check", ppg_sample_rate_check)
        ppg_hp = np.ravel(ppg_df["signal_filtered"])
        ppg_hp = signal.resample_poly(ppg_hp, 10, 12)
        #raise "PPG missmatch"
    
    ecg_sample_rate = float(len(ecg_data[' V5'])/10)
    #print("ecg_sample_rate",ecg_sample_rate)
    out = ecg.ecg(signal=ecg_data[' V5'], sampling_rate=ecg_sample_rate, show=False)

    # Determine rpeak_times
    rpeak_times = out['rpeaks']/ecg_sample_rate
    
    if ppg_sample_rate_check == ppg_sample_rate:
        ppg_hp = np.ravel(ppg_df["signal_filtered"])
        
        
    ppg_hp = signal.resample_poly(ppg_hp, 20, 2)
    #ppg_hp =  np.delete(ppg_hp, range(200), axis=0)
    
    # Determine correlation
    # Create ECG impulse signal for sync (at optical sample rate)
    opt_imp = np.zeros(int(ecg_data.shape[0] / ecg_sample_rate * ppg_upsample_rate))
    #display(opt_imp)
    opt_imp[(rpeak_times * ppg_upsample_rate).astype(int)] = 1
    #display(opt_imp)
    # remove mean and std of impulse signal
    opt_imp -= np.mean(opt_imp)
    opt_imp /= np.std(opt_imp)

    corr = []
    N = len(opt_imp)
        
    for idx in range(N, len(ppg_hp)):

        ppg_window = ppg_hp[idx - N:idx].copy()

        # remove mean and std of window
        ppg_window -= np.mean(ppg_window)
        ppg_window /= np.std(ppg_window)

        corr.append(np.dot(ppg_window, opt_imp))
    
    EXPAND_PPG = int(ppg_upsample_rate / 4) 
    # Plot the best correlation for a sanity check
    idx = range(N, len(ppg_hp))[np.argmax(corr)]
    
    ppg_window = ppg_hp[max(0,idx - N - EXPAND_PPG) : min(len(ppg_hp),idx + EXPAND_PPG)].copy()
    #print("idx",idx,N,len(ppg_window))

        

    # remove mean and std of window
    ppg_window -= np.mean(ppg_window)
    ppg_window /= np.std(ppg_window)
    ###############################
    #uncomment to see the plots
    #if row['Coordinator'] == "MB":
    fig, ax = plt.subplots(figsize=(30,10))
    ecg_downsample = signal.resample_poly(ecg_data.I, 20*500/ecg_sample_rate,10)
    ppg_upsample = ppg_window*100

    ax.plot(ecg_downsample)
    ax.plot(np.arange(len(ppg_upsample)) - EXPAND_PPG,ppg_upsample)
    fig.canvas.set_window_title( str(row['user_id']) + str(row.ecg_diagnosis))


        #ax.plot(opt_imp)
        #ax.plot(ppg_window)
        #fig, ax = plt.subplots()

    ###############################
    
    #Determine RR Correlation
    rpeaks = np.ravel(np.where(opt_imp>1))
    #display(rpeaks)
    #display(len(rpeaks))
    
    return ppg_window, rpeaks, ppg_df

In [7]:
def read_ppg_ecg(ppg_window, rpeaks, ppg_df):
    corro = []
    constant_array = []
    constant_array_2 = []
    
    for idx in range(0, len(rpeaks)):
        if idx == 0:
            constant = rpeaks[idx+1] 
            a = rpeaks[idx] - constant/4
            b = rpeaks[idx] + constant/2
            firstslice = ppg_window[a:b]
            try: 
                itemindex = np.where(ppg_window==max(firstslice))
            except:
                print("Empty sequence found")
                itemindex = [0]
            corro.append(itemindex[0])
            constant_array.append(a)
            constant_array_2.append(b)
        elif idx < (len(rpeaks)-1):
            constant1 = rpeaks[idx] - rpeaks[idx-1] 
            constant2 = rpeaks[idx+1] - rpeaks[idx] 
            a = rpeaks[idx] - constant1/4
            b = rpeaks[idx] + constant2/4
            secondslice = ppg_window[a:b]
            itemindex = np.where(ppg_window==max(secondslice))
            corro.append(itemindex[0])
            constant_array.append(a)
            constant_array_2.append(b)
        else:
            constant = rpeaks[idx] - rpeaks[idx-1] 
            a = rpeaks[idx] - constant/3
            b = rpeaks[idx] + constant/3
            thirdslice = ppg_window[a:b]
            itemindex = np.where(ppg_window==max(thirdslice))
            corro.append(itemindex[0])
            constant_array.append(a)
            constant_array_2.append(b)
            
    index = [str(i) for i in range(1, len(rpeaks)+1)]

    #df4.loc[len(df4)] = [row['user_id'],df3['difference'].mean(),ppg_df['bpm'],row['VentricularRate'],abs(row['VentricularRate']-ppg_df['bpm']),row['Diagnosis'],row['tachyarythmia'],row['normal']]
    
    ecg_rr_intervals = []
    for idx in range(0, len(rpeaks)):
        if idx > 0:
            itemindex =  rpeaks[idx] - rpeaks[idx-1]
            ecg_rr_intervals.append(itemindex)
            
    ppg_rr_intervals = []
    for idx in range(0, len(corro)):
         if idx > 0:
            itemindex = corro[idx] - corro[idx-1]
            ppg_rr_intervals.append(itemindex[0])
            
            
    index = [str(i) for i in range(1, len(ecg_rr_intervals)+1)]
    df1 = pd.DataFrame(ecg_rr_intervals, index=index)
    df2 = pd.DataFrame(ppg_rr_intervals, index=index)
    df3 = pd.concat([df1,df2],axis=1)
    df3['difference'] = abs(df2 - df1)
    df3['hr_difference'] = abs(60000/df2 - 60000/df1)
    df3['hr_ecg_signal'] = abs(60000/df1)
    df3['hr_ppg_signal'] = abs(60000/df2)
    df3 = df3.iloc[1:]
    display(df3)
    display(df3['difference'].median())
    
    
    for point in constant_array:
        ax.axvline(point,color="black",alpha=0.4)
        
    for point in constant_array_2:
        ax.axvline(point,color="red",alpha=0.4)
 
    return ecg_rr_intervals, ppg_rr_intervals, ppg_df, df3
    

In [8]:
def read_ppg_ecg_using_new_algorithm(ppg_window, rpeaks, ppg_df):

    corro = []
    for idx in range(0, len(rpeaks)):
        EXPAND_PPG = int(ppg_upsample_rate / 4) 
        EXPAND_SEARCH = int(0 * ppg_upsample_rate)
        ADVANCE = int(0 * ppg_upsample_rate)

        if idx == 0:
            a = max(0,rpeaks[idx] - (rpeaks[idx+1] - rpeaks[idx]) + EXPAND_PPG + ADVANCE)
            b = int(rpeaks[idx] + EXPAND_SEARCH) + EXPAND_PPG
        elif idx < (len(rpeaks)-1):
            a = int(rpeaks[idx-1] + ADVANCE) + EXPAND_PPG
            b = int(rpeaks[idx] + EXPAND_SEARCH) + EXPAND_PPG
        else:
            a = int(rpeaks[idx-1] + ADVANCE) + EXPAND_PPG
            b = int(rpeaks[idx] + EXPAND_SEARCH) + EXPAND_PPG
        
        #print(a,b,idx,len(ppg_window),len(ppg_hp))
        
        commonslice = ppg_window[a:b]
        turning_point = np.where(commonslice[:-1] > commonslice[1:])[-1][-1]
#         print("turning_point",turning_point)
        
        m_slice = np.min(commonslice)
        M_slice = np.max(commonslice)
        slice_index = np.arange(0,len(commonslice))[commonslice < ( m_slice + (M_slice-m_slice)*0.2 ) ]

        itemindex = a + slice_index[-1:] - EXPAND_PPG 
         #itemindex = np.where(ppg_window==min(secondslice))[0]
        corro.append(itemindex)
        
            
    #display(corro)
    index = [str(i) for i in range(1, len(rpeaks)+1)]
    ecg_rr_intervals = []
    ppg_rr_intervals = []         
    
    for idx in range(0, len(rpeaks)):
        if idx > 0:
            itemindex =  rpeaks[idx] - rpeaks[idx-1]
            ecg_rr_intervals.append(itemindex)
    for idx in range(0, len(corro)):
         if idx > 0:
            itemindex = corro[idx] - corro[idx-1]
            ppg_rr_intervals.append(itemindex[0])
            
            

    index = [str(i) for i in range(1, len(ecg_rr_intervals)+1)]
    df1 = pd.DataFrame(ecg_rr_intervals, index=index)
    df2 = pd.DataFrame(ppg_rr_intervals, index=index)
    df3 = pd.concat([df1,df2],axis=1)
    df3['difference'] = abs(df2 - df1)
    df3['hr_difference'] = abs(60000/df2 - 60000/df1)
    df3['hr_ecg_signal'] = abs(60000/df1)
    df3['hr_ppg_signal'] = abs(60000/df2)
    df3 = df3.iloc[1:]
    #for point in corro:
    #    ax.axvline(point,color="b",alpha=0.4)
        
    #for point in rpeaks:
    #    ax.axvline(point,color="r",alpha=0.4)
    
    
    display(df3)
    display(df3['difference'].median())
    #display(fig)
    
    return ecg_rr_intervals, ppg_rr_intervals, ppg_df, df3

In [9]:
display(azumio_ecg_mrn_merged['mrn'].nunique())
display(len(azumio_ecg_mrn_merged))

45

48

In [10]:
for index, row in azumio_ecg_mrn_merged.iterrows():
    filepath = 'rawdata/' + str(int(row['user_id'])) + ".zip"
    exists = os.path.isfile(filepath)
    azumio_ecg_mrn_merged.loc[index,'file_exists'] = exists

In [11]:
azumio_ecg_mrn_merged.columns

Index([u'UUID', u'filename', u'mrn', u'date_x', u'time_x', u'SystolicBP', u'DiastolicBP', u'VentricularRate', u'AtrialRate', u'PRInterval', u'QRSDuration', u'QTInterval', u'QTCorrected', u'PAxis', u'RAxis', u'TAxis', u'QRSCount', u'QOnset', u'QOffset', u'POnset', u'POffset', u'TOffset', u'ECGSampleBase', u'ECGSampleExponent', u'QTcFrederica680b0bba-1a03-49a4-90d3-c83db49b07a6', u'MUSE_20180702_200020_10000.xml', u'048819885', u'05-24-2018', u'13:15:21', u'NaN', u'NaN.1', u'63', u'63.1', u'170', u'76', u'430', u'440', u'12', u'27', u'13', u'11', u'218', u'256', u'133', u'183', u'433', u'500', u'0', u'437', u'user_id', u'date_y', u'time_y', u'Coordinator', u'IHR_Measurement', u'Ring_Measurement', u'tachyarythmia', u'normal', u'sent_to_robert', u'cardiac_ultrasound_done', u'notes', u'downloaded_azumio', u'download_ecg', u'ecg_done', u'afib', u'a_flutter', u'pvc', u'ecg_diagnosis', u'age', u'dob', u'gender (0 = male; 1 = female)', u'hypertension', u'prediabetes ', u'diabetes',
       u'cho

In [12]:
azumio_ecg_mrn_merged[['gender (0 = male; 1 = female)','tachyarythmia', 'normal', 'afib','a_flutter','pvc','hypertension','diabetes','cholesterol','prior_heart_attack','stroke','congestive_heart_failure','pacemaker','chronic_kidney_disease','atrial_fibrillation','atrial_flutter','sleep_apnea']].apply(pd.Series.value_counts).to_csv('azumio_described_categorical.csv')

In [13]:
azumio_ecg_mrn_merged = azumio_ecg_mrn_merged.loc[azumio_ecg_mrn_merged['file_exists'] == True]

In [14]:
azumio_ecg_mrn_merged = azumio_ecg_mrn_merged.loc[azumio_ecg_mrn_merged['afib'] == 1.0]

In [16]:
#azumio_ecg_mrn_merged = azumio_ecg_mrn_merged.loc[azumio_ecg_mrn_merged['Coordinator'] == 'TC']

In [17]:
#azumio_ecg_mrn_merged2 = azumio_ecg_mrn_merged.loc[azumio_ecg_mrn_merged['user_id'] == 262634]

In [16]:
ppg_sample_rate = 100.0
ppg_upsample_rate = 1000.0

col_names =  ['user_id', 'average_ms', 'rr_ecg', 'rr_ppg', 'bpm_ecg', 'bpm_ppg', 'bpm_difference','abs_bpm_difference', 'avg_bpm_difference_beat', 'hr_ecg_signal', 'hr_ppg_signal', 'diagnosis', 'tachyarythmia', 'afib', 'a_flutter', 'pvc', 'normal', 'coordinator', 'algorithm']
df5  = pd.DataFrame(columns = col_names)

for index, row in azumio_ecg_mrn_merged.iterrows():
    
    filepath = 'rawdata/' + str(int(row['user_id'])) + ".zip"
    ppg_path = os.path.expanduser(filepath)
    ecg_path = 'azumio_conv/' + str(row['UUID'] + '.csv')
    ppg_window, rpeaks, ppg_df = read_signals(ppg_path, ecg_path, ppg_sample_rate, ppg_upsample_rate)
    
    print(row['user_id'], "Loading data")
    ecg_rr_intervals, ppg_rr_intervals, ppg_df, average_signal = read_ppg_ecg_using_new_algorithm(ppg_window, rpeaks, ppg_df)
    ecg_rr_intervals_2, ppg_rr_intervals_2, ppg_df_2, average_signal_2 = read_ppg_ecg(ppg_window, rpeaks, ppg_df)
    
    if (average_signal_2['difference'].median() < average_signal['difference'].median()):
        ##If RR Interval is too sparse, use the old algorithm
        print(row['user_id'], "Using old algorithm")
        index = [str(i) for i in range(1, len(ecg_rr_intervals_2)+1)]
        df1 = pd.DataFrame(ecg_rr_intervals_2, index=index)
        df2 = pd.DataFrame(ppg_rr_intervals_2, index=index)

        df5.loc[len(df5)] = [row['user_id'],average_signal_2['difference'].median() ,df1.mean(), df2.mean(), row['VentricularRate'],ppg_df_2['bpm'],ppg_df_2['bpm']-row['VentricularRate'],abs(row['VentricularRate']-ppg_df_2['bpm']), average_signal_2['hr_difference'].median(), average_signal_2['hr_ecg_signal'].median(), average_signal_2['hr_ppg_signal'].median(), row['ecg_diagnosis'],row['tachyarythmia'],row['afib'],row['a_flutter'],row['pvc'],row['normal'],row['Coordinator'],0]
    else:
        print(row['user_id'], "Using new algorithm")
        index = [str(i) for i in range(0, (len(ecg_rr_intervals)))]
        df1 = pd.DataFrame(ecg_rr_intervals, index=index)
        df2 = pd.DataFrame(ppg_rr_intervals, index=index)


        df5.loc[len(df5)] = [row['user_id'],average_signal['difference'].median(),df1.mean(), df2.mean(), row['VentricularRate'],ppg_df['bpm'],ppg_df['bpm']-row['VentricularRate'],abs(row['VentricularRate']-ppg_df['bpm']), average_signal['hr_difference'].median(), average_signal['hr_ecg_signal'].median(), average_signal['hr_ppg_signal'].median(), row['ecg_diagnosis'],row['tachyarythmia'],row['afib'],row['a_flutter'],row['pvc'],row['normal'],row['Coordinator'],1]


/Users/papirobbi/anaconda3/lib/python2.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/Users/papirobbi/anaconda3/lib/python2.7/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/Users/papirobbi/anaconda3/lib/python2.7/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this w

(239173, 'Loading data')


,0,0,difference,hr_difference,hr_ecg_signal,hr_ppg_signal
2,908,798,110,9.108675,66.079295,75.187970
3,776,770,6,0.602490,77.319588,77.922078
4,540,593,53,9.930673,111.111111,101.180438
5,996,945,51,3.251100,60.240964,63.492063
6,768,785,17,1.691879,78.125000,76.433121
7,768,772,4,0.404793,78.125000,77.720207
8,768,782,14,1.398657,78.125000,76.726343
9,772,758,14,1.435466,77.720207,79.155673
10,776,623,153,18.988599,77.319588,96.308186
11,768,904,136,11.753319,78.125000,66.371681


34.0

,0,0,difference,hr_difference,hr_ecg_signal,hr_ppg_signal
2,908,875,33,2.492133,66.079295,68.571429
3,776,717,59,6.362421,77.319588,83.682008
4,540,615,75,13.550136,111.111111,97.560976
5,996,978,18,1.108729,60.240964,61.349693
6,768,768,0,0.000000,78.125000,78.125000
7,768,768,0,0.000000,78.125000,78.125000
8,768,769,1,0.101593,78.125000,78.023407
9,772,773,1,0.100544,77.720207,77.619664
10,776,675,101,11.569301,77.319588,88.888889
11,768,901,133,11.532325,78.125000,66.592675


...

,0,0,difference,hr_difference,hr_ecg_signal,hr_ppg_signal
2,950,872,78,5.649445,63.157895,68.807339
3,840,803,37,3.291229,71.428571,74.719801
4,1242,1345,103,3.699513,48.309179,44.609665
5,958,743,215,18.123221,62.630480,80.753701
6,934,1019,85,5.358573,64.239829,58.881256
7,920,887,33,2.426352,65.217391,67.643743
8,1234,841,393,22.721272,48.622366,71.343639
9,652,1266,614,44.631175,92.024540,47.393365


94.0

,0,0,difference,hr_difference,hr_ecg_signal,hr_ppg_signal
2,950,923,27,1.847522,63.157895,65.005417
3,840,785,55,5.004550,71.428571,76.433121
4,1242,1326,84,3.060310,48.309179,45.248869
5,958,790,168,13.318887,62.630480,75.949367
6,934,1028,94,5.874070,64.239829,58.365759
7,920,882,38,2.809820,65.217391,68.027211
8,1234,830,404,23.666790,48.622366,72.289157
9,652,1146,494,39.668519,92.024540,52.356021


89.0

(332112, 'Using old algorithm')


In [20]:
#df5 = df5.drop_duplicates(subset=['user_id'], keep='last').reset_index()
df5['average_ms'] = df5['average_ms'].astype(float)
#display(df4)
display(df5['average_ms'].mean())
display(df5['average_ms'].std())
display(df5.describe())

28.21590909090909

47.688109498347984

,average_ms,bpm_ppg,bpm_difference,abs_bpm_difference,avg_bpm_difference_beat,hr_ecg_signal,hr_ppg_signal,tachyarythmia,a_flutter,pvc,normal
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,41.000000,41.000000,4.0,41.000000
mean,28.215909,75.043956,0.998502,5.796561,3.209295,73.998491,73.738177,0.146341,0.024390,1.0,0.463415
std,47.688109,19.971013,8.782849,6.616428,5.705236,19.073076,18.951505,0.357839,0.156174,0.0,0.504854
min,0.000000,47.724422,-22.866212,0.163626,0.000000,49.423930,48.703352,0.000000,0.000000,1.0,0.000000
25%,3.000000,61.309274,-0.966593,1.423670,0.235478,60.000000,59.985015,0.000000,0.000000,1.0,0.000000
50%,12.500000,67.339210,1.495992,2.649799,0.728082,71.399317,71.259812,0.000000,0.000000,1.0,0.000000
75%,26.375000,84.228007,2.694987,8.278878,2.342960,79.849162,79.876741,0.000000,0.000000,1.0,1.000000
max,245.000000,121.914051,22.914051,22.914051,22.471608,120.000000,117.647059,1.000000,1.000000,1.0,1.000000


In [56]:
display(df6.head(n=10))

,user_id,average_ms,rr_ecg,rr_ppg,bpm_ecg,bpm_ppg,bpm_difference,abs_bpm_difference,avg_bpm_difference_beat,hr_ecg_signal,hr_ppg_signal,diagnosis,tachyarythmia,afib,a_flutter,pvc,normal,coordinator,algorithm
4,239173,25.5,0 768.727273 dtype: float64,0 769.454545 dtype: float64,78,77.251712,-0.748288,0.748288,1.800431,77.922604,78.074204,A Fib with premature aberrantly conducted comp...,1.0,1,0.0,NaN,0.0,MB,0
18,241277,108.0,0 761.0 dtype: float64,0 763.166667 dtype: float64,85,98.294305,13.294305,13.294305,14.110476,85.959885,84.269663,Abnormal record/ sinus rhythm wit hfrequent an...,0.0,0,0.0,1.0,0.0,MB,1
20,239184,15.0,0 621.142857 dtype: float64,0 657.357143 dtype: float64,99,121.914051,22.914051,22.914051,1.920123,120.000000,116.959064,Atrial Flutter w/ variable A-V block/ incomple...,1.0,0,1.0,NaN,0.0,MB,1
21,142247,53.5,0 806.0 dtype: float64,0 816.636364 dtype: float64,74,55.886421,-18.113579,18.113579,4.690400,73.892074,73.082041,Sinus rhythm with occasional premature ventric...,0.0,0,0.0,1.0,1.0,MB,1
26,255424,63.0,0 586.0 dtype: float64,0 573.785714 dtype: float64,98,117.870956,19.870956,19.870956,17.728856,100.000000,101.010101,"Afib; nonspecific ST abnormality, probably dig...",1.0,1,0.0,NaN,0.0,Ryan,1
27,255424,63.0,0 586.0 dtype: float64,0 573.785714 dtype: float64,98,117.870956,19.870956,19.870956,17.728856,100.000000,101.010101,"Afib; nonspecific ST abnormality, probably dig...",1.0,1,0.0,NaN,0.0,Ryan,1
32,230351,245.0,0 717.166667 dtype: float64,0 714.916667 dtype: float64,91,80.484097,-10.515903,10.515903,22.471608,90.909091,87.976540,Persistent atrial fibrillation,0.0,1,0.0,NaN,0.0,Ryan,0
34,258931,22.0,0 1319.333333 dtype: float64,0 1324.666667 dtype: float64,53,61.611764,8.611764,8.611764,0.467634,50.335570,50.083472,"Demand pacemaker, interpretation is based on i...",0.0,0,0.0,1.0,0.0,Ryan,0
39,258897,5.0,0 1137.75 dtype: float64,0 1137.75 dtype: float64,53,58.384530,5.384530,5.384530,0.244556,58.139535,58.422590,Atrial fibrillation with slow ventricular resp...,0.0,1,0.0,NaN,0.0,Ryan,1
42,29280,177.0,0 778.545455 dtype: float64,0 790.272727 dtype: float64,80,57.133788,-22.866212,22.866212,14.388889,71.028299,67.967256,Sinus rhythm with sinus arrhythmia with freque...,NaN,0,NaN,1.0,NaN,TC,0


In [38]:
df6 = df5.loc[(df5['afib'] == 1) | (df5['pvc'] == 1) | (df5['a_flutter'] == 1)]
df6.to_csv('correlation_irregular_rhythms.csv')
df7 = df5[~df5['user_id'].isin(df6['user_id'])]
df7.to_csv('correlation_regular_rhythms.csv')

In [49]:
display(df6.head(n=5))

,user_id,average_ms,rr_ecg,rr_ppg,bpm_ecg,bpm_ppg,bpm_difference,abs_bpm_difference,avg_bpm_difference_beat,hr_ecg_signal,hr_ppg_signal,diagnosis,tachyarythmia,afib,a_flutter,pvc,normal,coordinator,algorithm
4,239173,25.5,0 768.727273 dtype: float64,0 769.454545 dtype: float64,78,77.251712,-0.748288,0.748288,1.800431,77.922604,78.074204,A Fib with premature aberrantly conducted comp...,1.0,1,0.0,NaN,0.0,MB,0
18,241277,108.0,0 761.0 dtype: float64,0 763.166667 dtype: float64,85,98.294305,13.294305,13.294305,14.110476,85.959885,84.269663,Abnormal record/ sinus rhythm wit hfrequent an...,0.0,0,0.0,1.0,0.0,MB,1
20,239184,15.0,0 621.142857 dtype: float64,0 657.357143 dtype: float64,99,121.914051,22.914051,22.914051,1.920123,120.000000,116.959064,Atrial Flutter w/ variable A-V block/ incomple...,1.0,0,1.0,NaN,0.0,MB,1
21,142247,53.5,0 806.0 dtype: float64,0 816.636364 dtype: float64,74,55.886421,-18.113579,18.113579,4.690400,73.892074,73.082041,Sinus rhythm with occasional premature ventric...,0.0,0,0.0,1.0,1.0,MB,1
26,255424,63.0,0 586.0 dtype: float64,0 573.785714 dtype: float64,98,117.870956,19.870956,19.870956,17.728856,100.000000,101.010101,"Afib; nonspecific ST abnormality, probably dig...",1.0,1,0.0,NaN,0.0,Ryan,1


In [55]:
#Intraclass correlation
from rpy2.robjects import DataFrame, FloatVector, IntVector
from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages('ICC')

r_icc = importr("ICC", lib_loc="/Library/Frameworks/R.framework/Versions/3.5/Resources/library")
df = DataFrame({"groups": FloatVector(df5['hr_ecg_signal']),
                "values": FloatVector(df5['hr_ppg_signal'])})
icc_res = r_icc.ICCbare("groups", "values", data=df)
icc_val = icc_res[0] # icc_val now holds the icc value

# check whether icc value equals reference value
display("For signal:", icc_val)

r_icc = importr("ICC", lib_loc="/Library/Frameworks/R.framework/Versions/3.5/Resources/library")
df = DataFrame({"groups": FloatVector(df5['bpm_ecg']),
                "values": FloatVector(df5['bpm_ppg'])})
icc_res = r_icc.ICCbare("groups", "values", data=df)
icc_val = icc_res[0] # icc_val now holds the icc value

# check whether icc value equals reference value
display("For HR:", icc_val)

'For signal:'

0.9998935490690185

'For HR:'

0.8987629517602225

In [53]:
cc = importr("ICC", lib_loc="/Library/Frameworks/R.framework/Versions/3.5/Resources/library")
df = DataFrame({"groups": FloatVector(df6['hr_ecg_signal']),
                "values": FloatVector(df6['hr_ppg_signal'])})
icc_res = r_icc.ICCbare("groups", "values", data=df)
icc_val = icc_res[0] # icc_val now holds the icc value

# check whether icc value equals reference value
display(icc_val)

df = DataFrame({"groups": FloatVector(df6['bpm_ecg']),
                "values": FloatVector(df6['bpm_ppg'])})
icc_res = r_icc.ICCbare("groups", "values", data=df)
icc_val = icc_res[0] # icc_val now holds the icc value

# check whether icc value equals reference value
display("For HR:", icc_val)

1.0

'For HR:'

0.9966368811087726

In [54]:
cc = importr("ICC", lib_loc="/Library/Frameworks/R.framework/Versions/3.5/Resources/library")
df = DataFrame({"groups": FloatVector(df7['hr_ecg_signal']),
                "values": FloatVector(df7['hr_ppg_signal'])})
icc_res = r_icc.ICCbare("groups", "values", data=df)
icc_val = icc_res[0] # icc_val now holds the icc value

# check whether icc value equals reference value
display(icc_val)

df = DataFrame({"groups": FloatVector(df7['bpm_ecg']),
                "values": FloatVector(df7['bpm_ppg'])})
icc_res = r_icc.ICCbare("groups", "values", data=df)
icc_val = icc_res[0] # icc_val now holds the icc value

# check whether icc value equals reference value
display("For HR:", icc_val)

0.9998536528296715

'For HR:'

0.8785557781357279

In [39]:
chooseCRANmirror()

NameError: name 'chooseCRANmirror' is not defined

In [ ]:
bland_altman_plot(df5['rr_ppg'], df5['rr_ecg'])
#plt.title('Bland-Altman Plot')
plt.ylim((-65.0,65.0))
plt.xlabel("Average PPG and ECG RR interval \n (ms)") and plt.ylabel("Difference (PPG RR interval - ECG RR interval) \n (ms)")
plt.subplots_adjust(bottom=0.15)
plt.show()

In [ ]:
df6 = df5.loc[df5['afib'] == 1.0]
scatterplot(df6['rr_ppg'], df6['rr_ecg'])
#plt.title('Bland-Altman Plot')


df7 = df5.loc[df5['a_flutter'] == 1.0]
scatterplot(df7['rr_ppg'], df7['rr_ecg'])
#plt.title('Bland-Altman Plot')


df8 = df5.loc[df5['pvc'] == 1.0]
scatterplot(df8['rr_ppg'], df8['rr_ecg'])
#plt.title('Bland-Altman Plot')

In [ ]:
bland_altman_plot(df5['hr_ppg_signal'], df5['hr_ecg_signal'])
#plt.title('Bland-Altman Plot')
plt.ylim((-5.0,5.0))
plt.xlabel("Average PPG and ECG Heart Rate \n (BPM)") and plt.ylabel("Difference in Heart Rate (PPG - ECG) \n (BPM)")
plt.subplots_adjust(bottom=0.15)

plt.show()

In [ ]:
df6 = df5.loc[df5['afib'] == 1.0]
scatterplot(df6['hr_ppg_signal'], df6['hr_ecg_signal'])
#plt.title('Bland-Altman Plot
Fi
gure
df7 = df5.loc[df5['a_flutter'] == 1.0]
scatterplot(df7['hr_ppg_signal'], df7['hr_ecg_signal'])
#plt.title('Bland-Altman Plot')


df8 = df5.loc[df5['pvc'] == 1.0]
scatterplot(df8['hr_ppg_signal'], df8['hr_ecg_signal'])
display(df7)
#plt.title('Bland-Altman Plot')

In [ ]:

df5 = df4.groupby(['normal']).describe().reset_index()


df1 = df5.loc[df5['normal'] == 0]
df2 = df5.loc[df5['normal'] == 1]


display(df5.head(n=5))
display(stats.kruskal(df1['bpm_difference'], df2['bpm_difference']))
stats.kruskal(df1['average_ms'], df2['average_ms'])

In [ ]:
df4.to_csv('df4.csv')

In [ ]:
df5 = df4.groupby(['tachyarythmia']).describe().reset_index()
display(df5.head(n=5))

df1 = df5.loc[df5['tachyarythmia'] == 0]
df2 = df5.loc[df5['tachyarythmia'] == 1]

print(stats.kruskal(df1['bpm_difference'], df2['bpm_difference']))
print(stats.kruskal(df1['average_ms'], df2['average_ms']))





In [ ]:

df5 = df4.groupby(['coordinator']).describe().reset_index()
display(df5)

df1 = df5.loc[df5['coordinator'] == 'MB']
df2 = df5.loc[df5['coordinator'] == 'NW']
df3 = df5.loc[df5['coordinator'] == 'Ryan']


print(stats.kruskal(df1['bpm_difference'], df2['bpm_difference']))
print(stats.kruskal(df1['average_ms'], df2['average_ms']))



## Multivariate analysis

In [ ]:
y = df4['average_ms']
est = smf.ols(formula='y ~ C(coordinator)', data=df4)
est2 = est.fit()
display(est2.summary())

## Signal description

rr_raw all detected RR intervals, with minimal filtering
rr_filtered ( a median filter to exclude outliers)

signal_raw - raw output of the camera
signal_fixed - removed artifacts from the camera settings changes and internal camera processes. (basically removed jumps in the signal)
signal_filtered - detrended and filtered. 

signal_raw has some fixed applied to the data. original is what we got uploaded from the device.(it was in a different format). original should also includes accelerator and color data.


In [ ]:
filename = os.path.expanduser("rawdata/312679.zip")

with open(filename,"rb") as f:
    d = f.read()
    ppg_df = zip_to_object(d)

ppg_sample_rate_check = int(np.round(1000.0/np.mean(ppg_df["timestamps"][1:]-ppg_df["timestamps"][:-1])))
display(ppg_sample_rate_check)
ppg_hp = np.ravel(ppg_df["signal_filtered"])
display(len(ppg_hp))
ppg_hp = signal.resample_poly(ppg_hp, 10, 12)
display(len(ppg_hp))
# Load ecg data


# Read Colums
ecg_data = pd.read_csv("azumio_conv/20805c58-1c33-4c93-922a-171329eb1179.csv", sep=",")
ecg_sample_rate = len(ecg_data[' III'])/10
display(ecg_data.head(n=5))

In [ ]:
display(ppg_df["timestamps"])
item = np.argmax(ppg_df["timestamps"] > (2000))

In [ ]:
display(len(ppg_df["signal_raw"]))
item = 2000/100*120
length = len(ppg_df['signal_filtered'])
ts=ppg_df['signal_filtered']
ts = ts[item:length,:]
display(item)
display(len(ts))
plt.plot(ts)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))


ax = plt.subplot(5,1,1)
plt.plot(ppg_df["timestamps"],ppg_df["signal_raw"])
plt.subplot(5,1,2,sharex=ax)
plt.plot(ppg_df["timestamps"],ppg_df["signal_fixed"])
plt.ylim((1,2.5))
plt.subplot(5,1,3,sharex=ax)
plt.plot(ppg_df["timestamps"],ppg_df["signal_filtered"])

for ts in ppg_df["rr_raw"][:,0]:
    plt.axvline(x = ts,color="green")

for ts in ppg_df["rr_filtered"][:,0]:
    plt.axvline(x = ts,color="g")


plt.subplot(5,1,4,sharex=ax)
plt.plot(ppg_df["rr_filtered"][:,0],ppg_df["rr_filtered"][:,1],"o")


plt.show()

for key in ["bpm","sdsd","rmssd"]:
    print(key, ppg_df[key])

print(json.dumps(ppg_df["metadata"]["deviceDetails"],indent=2))


In [ ]:
hp_cut = 20.0 / 60.0
order = 71
a = signal.firwin(order, cutoff=hp_cut / (ppg_sample_rate / 2), window="hanning")
a = -a
a[int(order / 2)] = a[int(order / 2)] + 1

# High pass the PPG signal
#ppg_hp = signal.filtfilt(a, [1], np.ravel(ppg_df["signal_fixed"]))
ppg_signal = np.ravel(ppg_df["signal_filtered"])
#Delete first 10 datapoints because signal is usually messed up at the beginning.
ppg_hp =  np.delete(ppg_signal, range(10), axis=0)
timestamps =  np.delete(ppg_df["timestamps"], range(10), axis=0)


plt.plot(timestamps,ppg_hp)
plt.show()

In [ ]:
len(ecg_data.I)
display(ppg_df['bpm'])

In [ ]:
# Get rpeak times
out = ecg.ecg(signal=ecg_data.I, sampling_rate=ecg_sample_rate, show=False)

#display(out)
rpeak_times = out['rpeaks'] / ecg_sample_rate
display(rpeak_times)

In [ ]:
# Create ECG impulse signal for sync (at optical sample rate)
display(ecg_sample_rate)
display(ppg_sample_rate)
opt_imp = np.zeros(int(ecg_data.shape[0] / ecg_sample_rate * ppg_sample_rate))
opt_imp[(rpeak_times * ppg_sample_rate).astype(int)] = 1

In [ ]:
# Determine correlation
# remove mean and std of impulse signal
opt_imp -= np.mean(opt_imp)
opt_imp /= np.std(opt_imp)

corr = []
N = len(opt_imp)
display(N)
display(len(ppg_hp))
for idx in range(N, len(ppg_hp)):
    ppg_window = ppg_hp[idx - N:idx].copy()
    
    # remove mean and std of window
    ppg_window -= np.mean(ppg_window)
    ppg_window /= np.std(ppg_window)
    
    corr.append(np.dot(ppg_window, opt_imp))
    

In [ ]:
plt.plot(corr)
display(np.argmax(corr))

In [ ]:
# Plot the best correlation for a sanity check
idx = range(N, len(ppg_hp))[np.argmax(corr)]
ppg_window = ppg_hp[idx - N:idx].copy()

# remove mean and std of window
ppg_window -= np.mean(ppg_window)
ppg_window /= np.std(ppg_window)

fig, ax = plt.subplots()
ax.plot(opt_imp)
ax.plot(ppg_window)

In [ ]:
display(len(opt_imp))
display(len(ppg_window))

In [ ]:
df = signal.find_peaks_cwt(ppg_window, ppg_df["rr_filtered"])

In [ ]:
rpeaks = np.ravel(np.where(opt_imp>1))
display(rpeaks)
display(len(rpeaks))
corro = []
for idx in range(0, len(rpeaks)):
    if idx == 0:
        a = rpeaks[idx] + 10
        firstslice = ppg_window[0:a]
        itemindex = np.where(ppg_window==max(firstslice))
        corro.append(itemindex)
    else:
        constant = rpeaks[idx] - rpeaks[idx-1] 
        a = rpeaks[idx-1] + constant/2
        b = rpeaks[idx] + constant/2
        secondslice = ppg_window[a:b]
        itemindex = np.where(ppg_window==max(secondslice))
        corro.append(itemindex)
display(corro)

In [ ]:
index = [str(i) for i in range(1, len(rpeaks)+1)]

df1 = pd.DataFrame(rpeaks, index=index)
df2 = pd.DataFrame(corro, index=index)
df3 = pd.concat([df1,df2],axis=1)
df3['difference'] = abs(df2 - df1)
df3.columns = ['ecg', 'ppg', 'difference']
display(df3)



In [ ]:
col_names =  ['user_id', 'average_ms']
df4  = pd.DataFrame(columns = col_names)
df4.loc[len(df4)] = ['239181',df3['difference'].mean()]
display(df4)

In [ ]:
# Create timestamps for the ecg signal
best_correlation = np.argmax(corr) #Where correlation is the best
display(best_correlation)


#Set start of the ECG in sync with timestamp on the PPG DF with best correlation
ecg_start_utc = ppg_df["timestamps"].iloc[best_correlation]
ecg_utc = np.asarray(range(ecg_data.shape[0])) / ecg_sample_rate + ecg_start_utc

In [ ]:
Avramlist(ecg_data.columns.values)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 1, sharex=True)

ax[0].plot(ppg_df["timestamps"],ppg_df["signal_filtered"])
ax[0].set_title('PPG Signal', fontsize=16)

#ppg_df_2 = 
ppg_window = ppg_df.pd1[(ppg_df.utc > ecg_start_utc) & (ppg_df.utc <  ecg_utc[-1])]
#ax[0].set_ylim([ppg_window.min(), ppg_window.max()])


ax[1].plot(ecg_utc, ecg_data[' V1'])
ax[1].set_title('ECG Signal', fontsize=16)

ax[1].set_xlim([ecg_start_utc - 20, ecg_utc[-1] + 20])
